In [81]:
from db import db
import pymongo
import time

one = next(db.repka_logs.aggregate([
    {'$sort': { 'time': -1 }},
    { '$match': { 'is_viewed': { "$ne": True } } },
    {'$limit': 1 }
]))

# text = [one["time"]]

# for one in db.repka_logs.find({ "session_id": one['session_id'] }).sort([("message_id", 1)]):
#     text.append(f"{one['request']}\n{one['response']}\n")

# print("\n".join(text))



# db.repka_logs.update({'version': 1}, {"$set": { 'time': 0 }} )
one

{'_id': ObjectId('61f6a4d40c7bd4004b788682'),
 'request': 'ежедневный пророк',
 'message_id': 4,
 'session_id': '3e87b1b8-b428-4e14-8c25-f891399b5289',
 'response': 'Я ежедневный пророк. Помогу вам. Ежедневный пророк за ежедневног',
 'time': 1643554004995.0,
 'version': 2}

In [82]:
# db.command({ 'collStats' : "repka_logs", 'scale': 1024*1024 })

In [83]:
from IPython.display import display
